In [1]:
%gui qt

In [1]:
config_file = "../code/agrid.py"
with open(config_file) as f:
    code = compile(f.read(), config_file, 'exec')
    exec(code, globals(), locals())

# ... or use you Python path or present working directory:
#from agrid import *

/Users/tobiasstal/anaconda3/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/tobiasstal/anaconda3/lib/python3.6/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
#Continental
ant = Grid(crs=3031, res = [10*km, 10*km], left = -3100*km, up=3100*km, right = 3100*km, down = -3100*km)

ant.ds['ICE'] = (('Y', 'X'), 
                ant.read_raster('../data/bedmap2_tiff/bedmap2_thickness.tif'))

no_data = 32767.
ant.ds['ICE'] = ant.ds['ICE'].where(ant.ds['ICE'] != no_data)  

In [ ]:
ant.map_grid('ICE', vmin=0, vmax=4500, cmap='viridis', cbar=True)

In [ ]:
dranage, int_map = ant.assign_shape('../data/GSFC_DrainageSystems.shp', 
                                                 'ID', map_to_int = True, return_map = True)
ant.ds['DRANAGE'] = (('Y', 'X'), dranage)

In [ ]:
ant.map_grid(ant.ds['DRANAGE'], cmap='viridis', cbar=True)

In [ ]:
polygons = [int_map[str(x) + 'g'] for x in range(2,18)]

In [ ]:
ant.ds['EAST_ICE'] = ant.ds['ICE']*ant.ds['DRANAGE'].isin(polygons)
ant.map_grid('EAST_ICE', cmap='viridis')

In [ ]:
print(int(ant.ds['EAST_ICE'].sum()*np.prod(ant.res)/km**3),'km3')

In [ ]:
import scipy

# prepare some interesting function:
def f(x, y):
    return 3.0*scipy.sin(x*y+1e-4)/(x*y+1e-4)

x = scipy.arange(-7., 7.05, 0.1)
y = scipy.arange(-5., 5.05, 0.1)

# 3D visualization of f:
from tvtk.tools import mlab
fig = mlab.figure()
s = mlab.SurfRegular(x, y, f)
fig.add(s)

In [ ]:
test_volume_slice()

In [ ]:
# Author: Gael Varoquaux <gael.varoquaux@normalesup.org>
# Copyright (c) 2008, Enthought, Inc.
# License: BSD Style.

# Retrieve the grand Canyon topological data ##################################
import os
# Original file:
#'ftp://e0srp01u.ecs.nasa.gov/srtm/version2/SRTM1/Region_04/N36W113.hgt.zip'
if not os.path.exists('N36W113.hgt.zip'):
    # Download the data
    try:
        from urllib import urlopen
    except ImportError:
        from urllib.request import urlopen
    print('Downloading data, please wait (10M)')
    opener = urlopen(
    'https://s3.amazonaws.com/storage.enthought.com/www/sample_data/N36W113.hgt.zip'
        )
    open('N36W113.hgt.zip', 'wb').write(opener.read())

# Load the data (signed 2 byte integers, big endian) ##########################
import zipfile
import numpy as np

data = np.fromstring(zipfile.ZipFile('N36W113.hgt.zip').read('N36W113.hgt'),
                    '>i2')
data.shape = (3601, 3601)
data = data.astype(np.float32)

# Plot an interesting section #################################################
from mayavi import mlab
data = data[:1000, 900:1900]
# Convert missing values into something more sensible.
data[data == -32768] = data[data > 0].min()

mlab.figure(size=(400, 320), bgcolor=(0.16, 0.28, 0.46))
mlab.surf(data, colormap='gist_earth', warp_scale=0.2,
            vmin=1200, vmax=1610)
# The data takes a lot of memory, and the surf command has created a
# copy. We free the inital memory.
del data

# A view of the canyon
mlab.view(-5.9, 83, 570, [5.3, 20, 238])
mlab.show()

In [7]:
from mayavi import mlab

ant.ds['BED'] = (('Y', 'X'), 
                ant.read_raster('../data/bedmap2_tiff/bedmap2_bed.tif'))

ant.ds['BED'] = ant.ds['BED'].where(ant.ds['BED'] != no_data)  

data = ant.ds['BED'].values
data.shape = ant.nn
data = data.astype(np.float32)

# Plot an interesting section #################################################


s = mlab.figure(size=(1000, 1000), bgcolor=(1, 1, 1))
s = mlab.surf(data, colormap='terrain', warp_scale=0.01,
            vmin=-4000, vmax=4000)

s.scene.off_screen_rendering = True
#scene = engine.scenes[0]
#scene.scene.show_axes = True
#scene.scene.light_manager.light_mode = 'vtk'
#scene.scene.isometric_view()
#array_source.origin = array([  0.,   0., 300.])

# A view of the canyon
#mlab.view(45, 10000, 5000, [0, 0, 0])
mlab.savefig('param10.png', figure = s, size=(500,500), magnification=10)

In [ ]:
ant.ds['BED']

In [7]:
from mayavi import mlab
s = mlab.test_plot3d()
s.scene.off_screen_rendering = True
mlab.savefig('para.png', figure = s)
s.scene.off_screen_rendering = False